# **Data Collection**

## Objectives

* Fetch data from Kaggle and prepare it for further processes.

## Inputs

*   Kaggle JSON file - the authentication token. 

## Outputs

* Generate Dataset: inputs/datasets/cherry_dataset

## Additional Comments | Insights | Conclusions


* No additional comments.


---

# Import packages

In [2]:
%pip install -r /workspace/mildew-detection-in-cherry-leaves/requirements.txt

     |████████████████████████████████| 17.1 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 84.8 MB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 32.1 MB/s eta 0:00:01
     |████████████████████████████████| 283 kB 96.8 MB/s eta 0:00:01
     |████████████████████████████████| 15.6 MB 59.2 MB/s eta 0:00:01
     |████████████████████████████████| 8.1 MB 63.0 MB/s eta 0:00:01
     |████████████████████████████████| 24.9 MB 65.6 MB/s eta 0:00:01
     |████████████████████████████████| 225.6 MB 36 kB/s s eta 0:00:01��██▋     | 187.5 MB 80.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 80.0 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 99.6 MB/s eta 0:00:01
     |████████████████████████████████| 813 kB 52.3 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 66.0 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 90.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 80.7 MB/

In [11]:
import numpy
import os

## Change the working directory

In [12]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection-in-cherry-leaves'

In [13]:
os.chdir('/workspace/mildew-detection-in-cherry-leaves')
print("You set a new current directory")

You set a new current directory


In [14]:

current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection-in-cherry-leaves'

# Install Kaggle

In [11]:
# install kaggle package
%pip install kaggle==1.5.12

     |████████████████████████████████| 58 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 4.4 MB/s eta 0:00:011
     |████████████████████████████████| 78 kB 16.9 MB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73049 sha256=5fe562f2f80acd0d34d9605e6f23853177fef18d5d4ee011b45d918e9ea5f28e
  Stored in directory: /workspace/.pyenv_mirror/pip_cache/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle
You should consider upgrading via the '/home/gitpod/.pyenv/versions/3.8.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


---

Run the cell below **to change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON**.

In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

* Get the dataset path from the [Kaggle URL](https://www.kaggle.com/datasets/codeinstitute/cherry-leaves). When you are viewing the dataset at Kaggle, check what is after https://www.kaggle.com/ (in some cases kaggle.com/datasets). You should copy that at KaggleDatasetPath.
* Set your destination folder.

Set the Kaggle Dataset and Download it.

In [5]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 98%|█████████████████████████████████████▎| 54.0M/55.0M [00:02<00:00, 36.4MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 26.2MB/s]


Unzip the downloaded file, and delete the zip file.

In [6]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

---

## Data cleaning

### Check and remove non-image files

In [7]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))


In [8]:
remove_non_image_file(my_data_dir='inputs/cherry_dataset/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


## Split train validation test set

In [9]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)


Conventionally,
* The training set is divided into a 0.70 ratio of data.
* The validation set is divided into a 0.10 ratio of data.
* The test set is divided into a 0.20 ratio of data.

In [10]:
split_train_validation_test_images(my_data_dir=f"inputs/cherry_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )


---

# Congratulations

---